<a href="https://colab.research.google.com/github/Alan-Marcus/HCCS/blob/main/03%20Analysis%20scripts%20and%20templates/SLEAP/Colab%20Notebooks/SLEAP_Movement_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HCCS: Movement Analysis for SLEAP**

This is an example notebook to calculate the movement parameters for up to five animals in videos recorded using the Home-Cage Camera System (HCCS; https://github.com/Alan-Marcus/HCCS) and tracked using SLEAP (https://sleap.ai).
This notebook will calculate the Total Distance Moved and Time Mobile for each animal at each time point and save them in a CSV file. Optionally, the raw movement of each animal can also be plotted.

#0. Setup

1. Upload the video analysis files and the video_details file to the analysis folder in Google Drive.
2. Run the following cells to setup the required utilities and connect your Google Drive to this session.

In [ ]:
#@title Install Utilities
import h5py
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from google.colab import files
from google.colab import drive
from tqdm import tqdm
import matplotlib.pyplot as plt
print('done')

In [ ]:
#@title Connect Google Drive
drive.mount('/content/drive/')

#1. Specify Settings and Calculate Movement Parameters

In [ ]:
#@title Edit Parameter Names (this step will overwrite the existing movement results file unless a different filename is specified here)
path_to_SLEAP_folder = "/content/drive/MyDrive/SLEAP" #@param {type:"string"}
video_details_file = "HCCS_example_project_video_details.csv"  #@param {type:"string"}
movement_results_filename = "HCCS_example_project-movement-results"  #@param {type:"string"}

#@markdown Choose missing value imputation method
missing_fill = "previous" #@param ["previous", "next", "linear"]

#@markdown Edit movement threshold value (cm; values <= this value are zeroed)
threshold_limit =  0.2#@param {type:"number"} 

#@markdown Choose to create raw movement plots
xy_movement = "Yes" #@param ["Yes", "No"]
xy_tracks = "Yes" #@param ["Yes", "No"]

#---don't edit below
#Note: if number of animals is greater than 5, edit code where indicated ("hard-coded for up to 5 animals")
path_to_SLEAP_analysis_folder = path_to_SLEAP_folder + "/analysis/"
movementresults = path_to_SLEAP_analysis_folder + movement_results_filename + ".csv"
h5_list = path_to_SLEAP_analysis_folder + video_details_file

#hard-coded for up to 5 animals
movement_parameters = np.array([['','','','Total distance moved (cm)','','','','','Time mobile (s)','','','','','Time mobile (%)']])
np.savetxt(movementresults, movement_parameters, delimiter=',', fmt='%s')
movement_parameters = np.array([['Date','Time_point','Group','Animal_1','Animal_2','Animal_3','Animal_4','Animal_5','Animal_1','Animal_2','Animal_3','Animal_4','Animal_5','Animal_1','Animal_2','Animal_3','Animal_4','Animal_5','Filename']])
with open(movementresults, "ab") as f:
    np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')

#Define filling of missing values
def fill_missing(Y, kind=missing_fill):
    """Fills missing values independently along each dimension after the first."""
    # Store initial shape.
    initial_shape = Y.shape
    # Flatten after first dim.
    Y = Y.reshape((initial_shape[0], -1))
    # Interpolate along each slice.
    for i in range(Y.shape[-1]):
        y = Y[:, i]
        # Build interpolant.
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)
        # Fill missing
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        # Fill leading or trailing NaNs with the nearest non-NaN values
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])
        # Save slice
        Y[:, i] = y
    # Restore to initial shape.
    Y = Y.reshape(initial_shape)
    return Y

#Load data, iterate over each row of data to fill missing values and calculate movement parameters
data = pd.read_csv(h5_list)
num_rows = [*range(len(data))]

for row in tqdm(num_rows):
  #Load data
  with h5py.File(path_to_SLEAP_analysis_folder + data.loc[row,"h5_filename"], "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]
  locations = fill_missing(locations)

  #Calculate movement parameters
  h5_filename = data.loc[row,"h5_filename"]
  vid_date = data.loc[row,"date"]
  timepoint = data.loc[row,"time_point"]
  group = data.loc[row,"group"]
  start_frame = int(data.loc[row,"start_frame"])
  end_frame = int(data.loc[row,"end_frame"])
  pixels_per_cm = data.loc[row,"pixels_per_cm"]
  frames_per_s = data.loc[row,"fps"]

  threshold = pixels_per_cm*threshold_limit
  frame_num = list(range(start_frame-1, end_frame))   #Note: SLEAP interface reports first frame as 1 not 0, so automatically adjusted here
  animal_list = list(range(0,locations.shape[3]))   #dynamiclly creates animal_list
  dict = {}

  for animal in animal_list:
    raw_distance = []
    filtered_distance = []
    time_mobile_frame = []

    for frame in frame_num:
      if frame >start_frame-1:
        calc_raw_distance = ((locations[frame, 0, 0, animal]-locations[frame-1, 0, 0, animal])**2+(locations[frame, 0, 1, animal]-locations[frame-1, 0, 1, animal])**2)**(1/2)
        if calc_raw_distance <= threshold:  #values <= to the threshold value are zeroed
          calc_filtered_distance = 0
        else:
          calc_filtered_distance = calc_raw_distance
        if calc_filtered_distance >0:
          time_mobile_frame.append(1)
        else:
          time_mobile_frame.append(0)
        raw_distance.append(calc_raw_distance)
        filtered_distance.append(calc_filtered_distance)

    dict['total_distance_animal' + str(animal+1)] = sum(filtered_distance) / pixels_per_cm
    dict['time_mobile_s_animal' + str(animal+1)] = sum(time_mobile_frame) / frames_per_s
    dict['time_mobile_pc_animal' + str(animal+1)] = sum(time_mobile_frame)*100 / (end_frame - start_frame)

  #appends calculated parameters in one row to existing csv file (hard-coded for up to 5 animals)
  if len(animal_list) == 5:
    movement_parameters = np.array([[vid_date, timepoint, group, dict['total_distance_animal1'], dict['total_distance_animal2'], dict['total_distance_animal3'], dict['total_distance_animal4'], dict['total_distance_animal5'], dict['time_mobile_s_animal1'],  dict['time_mobile_s_animal2'], dict['time_mobile_s_animal3'], dict['time_mobile_s_animal4'], dict['time_mobile_s_animal5'], dict['time_mobile_pc_animal1'], dict['time_mobile_pc_animal2'], dict['time_mobile_pc_animal3'], dict['time_mobile_pc_animal4'], dict['time_mobile_pc_animal5'], h5_filename]])
    with open(movementresults, "ab") as f:
        np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')
  elif len(animal_list) == 4:
    movement_parameters = np.array([[vid_date, timepoint, group, dict['total_distance_animal1'], dict['total_distance_animal2'], dict['total_distance_animal3'], dict['total_distance_animal4'], "", dict['time_mobile_s_animal1'],  dict['time_mobile_s_animal2'], dict['time_mobile_s_animal3'], dict['time_mobile_s_animal4'], "", dict['time_mobile_pc_animal1'], dict['time_mobile_pc_animal2'], dict['time_mobile_pc_animal3'], dict['time_mobile_pc_animal4'], "", h5_filename]])
    with open(movementresults, "ab") as f:
        np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')
  elif len(animal_list) == 3:
    movement_parameters = np.array([[vid_date, timepoint, group, dict['total_distance_animal1'], dict['total_distance_animal2'], dict['total_distance_animal3'], "", "", dict['time_mobile_s_animal1'],  dict['time_mobile_s_animal2'], dict['time_mobile_s_animal3'], "", "", dict['time_mobile_pc_animal1'], dict['time_mobile_pc_animal2'], dict['time_mobile_pc_animal3'], "", "", h5_filename]])
    with open(movementresults, "ab") as f:
        np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')
  elif len(animal_list) == 2:
    movement_parameters = np.array([[vid_date, timepoint, group, dict['total_distance_animal1'], dict['total_distance_animal2'], "", "", "", dict['time_mobile_s_animal1'],  dict['time_mobile_s_animal2'], "", "", "", dict['time_mobile_pc_animal1'], dict['time_mobile_pc_animal2'], "", "", "", h5_filename]])
    with open(movementresults, "ab") as f:
        np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')
  elif len(animal_list) == 1:
    movement_parameters = np.array([[vid_date, timepoint, group, dict['total_distance_animal1'], "", "", "", "", dict['time_mobile_s_animal1'],  "", "", "", "", dict['time_mobile_pc_animal1'], "", "", "", "", h5_filename]])
    with open(movementresults, "ab") as f:
        np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')
  elif len(animal_list) == 0:
    movement_parameters = np.array([[vid_date, timepoint, group, "no animals!?", "", "", "", "", "",  "", "", "", "", "", "", "", "", "", h5_filename]])
    with open(movementresults, "ab") as f:
        np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')
  elif len(animal_list) > 5:
    movement_parameters = np.array([[vid_date, timepoint, group, ">5 animals!", "", "", "", "", "",  "", "", "", "", "", "", "", "", "", h5_filename]])
    with open(movementresults, "ab") as f:
        np.savetxt(f, movement_parameters, delimiter=',', fmt='%s')

  #Plot centroid xy individual movement and xy tracks (hard-coded for up to 5 animals)
  centroid_loc = locations[start_frame-1:end_frame, 0, :, :]/pixels_per_cm
  if len(animal_list) == 5:
    if xy_movement == "Yes":
      #xy individual movement
      plt.figure(figsize=(15,6))
      plt.ylim(-972/pixels_per_cm,1296/pixels_per_cm)
      plt.title('Centroid xy movement: '+h5_filename+'\n(x-axis shown >0; y-axis shown <0)')
      #x movement
      plt.plot(centroid_loc[:,0,0], 'b', label='animal-1')
      plt.plot(centroid_loc[:,0,1], 'r', label='animal-2')
      plt.plot(centroid_loc[:,0,2], 'y', label='animal-3')
      plt.plot(centroid_loc[:,0,3], 'm', label='animal-4')
      plt.plot(centroid_loc[:,0,4], 'g', label='animal-5')
      #y movement
      plt.plot(-1*centroid_loc[:,1,0], 'b')
      plt.plot(-1*centroid_loc[:,1,1], 'r')
      plt.plot(-1*centroid_loc[:,1,2], 'y')
      plt.plot(-1*centroid_loc[:,1,3], 'm')
      plt.plot(-1*centroid_loc[:,1,4], 'g')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-movement_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
    if xy_tracks == "Yes":
      #xy tracks
      plt.figure(figsize=(8,6))
      plt.xlim(0,1296/pixels_per_cm)
      plt.ylim(972/pixels_per_cm,0)
      plt.title('Centroid xy tracks: '+h5_filename)
      plt.plot(centroid_loc[:,0,0],centroid_loc[:,1,0], 'b',label='animal-1')
      plt.plot(centroid_loc[:,0,1],centroid_loc[:,1,1], 'r',label='animal-2')
      plt.plot(centroid_loc[:,0,2],centroid_loc[:,1,2], 'y',label='animal-3')
      plt.plot(centroid_loc[:,0,3],centroid_loc[:,1,3], 'm',label='animal-4')
      plt.plot(centroid_loc[:,0,4],centroid_loc[:,1,4], 'g',label='animal-5')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-tracks_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
  elif len(animal_list) == 4:
    if xy_movement == "Yes":
      #xy individual movement
      plt.figure(figsize=(15,6))
      plt.ylim(-972/pixels_per_cm,1296/pixels_per_cm)
      plt.title('Centroid xy movement: '+h5_filename+'\n(x-axis shown >0; y-axis shown <0)')
      #x movement
      plt.plot(centroid_loc[:,0,0], 'b', label='animal-1')
      plt.plot(centroid_loc[:,0,1], 'r', label='animal-2')
      plt.plot(centroid_loc[:,0,2], 'y', label='animal-3')
      plt.plot(centroid_loc[:,0,3], 'm', label='animal-4')
      #y movement
      plt.plot(-1*centroid_loc[:,1,0], 'b')
      plt.plot(-1*centroid_loc[:,1,1], 'r')
      plt.plot(-1*centroid_loc[:,1,2], 'y')
      plt.plot(-1*centroid_loc[:,1,3], 'm')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-movement_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
    if xy_tracks == "Yes":
      #xy tracks
      plt.figure(figsize=(8,6))
      plt.xlim(0,1296/pixels_per_cm)
      plt.ylim(972/pixels_per_cm,0)
      plt.title('Centroid xy tracks: '+h5_filename)
      plt.plot(centroid_loc[:,0,0],centroid_loc[:,1,0], 'b',label='animal-1')
      plt.plot(centroid_loc[:,0,1],centroid_loc[:,1,1], 'r',label='animal-2')
      plt.plot(centroid_loc[:,0,2],centroid_loc[:,1,2], 'y',label='animal-3')
      plt.plot(centroid_loc[:,0,3],centroid_loc[:,1,3], 'm',label='animal-4')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-tracks_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
  elif len(animal_list) == 3:
    if xy_movement == "Yes":
      #xy individual movement
      plt.figure(figsize=(15,6))
      plt.ylim(-972/pixels_per_cm,1296/pixels_per_cm)
      plt.title('Centroid xy movement: '+h5_filename+'\n(x-axis shown >0; y-axis shown <0)')
      #x movement
      plt.plot(centroid_loc[:,0,0], 'b', label='animal-1')
      plt.plot(centroid_loc[:,0,1], 'r', label='animal-2')
      plt.plot(centroid_loc[:,0,2], 'y', label='animal-3')
      #y movement
      plt.plot(-1*centroid_loc[:,1,0], 'b')
      plt.plot(-1*centroid_loc[:,1,1], 'r')
      plt.plot(-1*centroid_loc[:,1,2], 'y')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-movement_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
    if xy_tracks == "Yes":
      #xy tracks
      plt.figure(figsize=(8,6))
      plt.xlim(0,1296/pixels_per_cm)
      plt.ylim(972/pixels_per_cm,0)
      plt.title('Centroid xy tracks: '+h5_filename)
      plt.plot(centroid_loc[:,0,0],centroid_loc[:,1,0], 'b',label='animal-1')
      plt.plot(centroid_loc[:,0,1],centroid_loc[:,1,1], 'r',label='animal-2')
      plt.plot(centroid_loc[:,0,2],centroid_loc[:,1,2], 'y',label='animal-3')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-tracks_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
  elif len(animal_list) == 2:
    if xy_movement == "Yes":
      #xy individual movement
      plt.figure(figsize=(15,6))
      plt.ylim(-972/pixels_per_cm,1296/pixels_per_cm)
      plt.title('Centroid xy movement: '+h5_filename+'\n(x-axis shown >0; y-axis shown <0)')
      #x movement
      plt.plot(centroid_loc[:,0,0], 'b', label='animal-1')
      plt.plot(centroid_loc[:,0,1], 'r', label='animal-2')
      #y movement
      plt.plot(-1*centroid_loc[:,1,0], 'b')
      plt.plot(-1*centroid_loc[:,1,1], 'r')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-movement_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
    if xy_tracks == "Yes":
      #xy tracks
      plt.figure(figsize=(8,6))
      plt.xlim(0,1296/pixels_per_cm)
      plt.ylim(972/pixels_per_cm,0)
      plt.title('Centroid xy tracks: '+h5_filename)
      plt.plot(centroid_loc[:,0,0],centroid_loc[:,1,0], 'b',label='animal-1')
      plt.plot(centroid_loc[:,0,1],centroid_loc[:,1,1], 'r',label='animal-2')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-tracks_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
  elif len(animal_list) == 1:
    if xy_movement == "Yes":
      #xy individual movement
      plt.figure(figsize=(15,6))
      plt.ylim(-972/pixels_per_cm,1296/pixels_per_cm)
      plt.title('Centroid xy movement: '+h5_filename+'\n(x-axis shown >0; y-axis shown <0)')
      #x movement
      plt.plot(centroid_loc[:,0,0], 'b', label='animal-1')
      #y movement
      plt.plot(-1*centroid_loc[:,1,0], 'b')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-movement_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
    if xy_tracks == "Yes":
      #xy tracks
      plt.figure(figsize=(8,6))
      plt.xlim(0,1296/pixels_per_cm)
      plt.ylim(972/pixels_per_cm,0)
      plt.title('Centroid xy tracks: '+h5_filename)
      plt.plot(centroid_loc[:,0,0],centroid_loc[:,1,0], 'b',label='animal-1')
      #plot legend
      plt.legend()
      #save plot to file
      plt.savefig(path_to_SLEAP_analysis_folder + "plots/xy-tracks_"+h5_filename+'.png', bbox_inches='tight')
      plt.close()
  elif len(animal_list) == 0:
    print("\n", "No animals detected in "+h5_filename)
  elif len(animal_list) > 5:
    print("\n", "More than 5 animals detected; edit underlying code to process "+h5_filename)

print()
print(len(data), "videos analysed and data added to " + movementresults)
if xy_movement == "Yes":
  print(len(data), "xy_movement plots created and saved in " + path_to_SLEAP_analysis_folder + "plots/")
if xy_tracks == "Yes":
  print(len(data), "xy_tracks plots created and saved in " + path_to_SLEAP_analysis_folder + "plots/")

#2. Download output

In [ ]:
#@title Download the movement results file
files.download(movementresults)

In [ ]:
#@title Zip the movement plots
!cd $path_to_SLEAP_analysis_folder/plots/ && zip -r $path_to_SLEAP_folder/$movement_results_filename-plots *.png
print("Done")

In [ ]:
#@title Download the zipped movement plots
files.download(path_to_SLEAP_folder + "/" + movement_results_filename + "-plots" + ".zip")